In [15]:
import random
import csv
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [16]:
variables = {
    "Weapon": ["Knife", "Gun", "Poison", "Fists"],
    "Location": ["Living Room", "Kitchen", "Bedroom", "Garden"],
    "Opportunity": ["High", "Medium", "Low"],
    "Motive": ["Jealousy", "Revenge", "Greed", "None"]
}

In [17]:
suspects = ["John", "Mary", "Peter", "Alice"]


In [18]:
def generate_data(num_data_points):
    with open("mystery_data.csv", "w", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["Case ID", "Suspect", "Variable", "Value", "Description", "Solution"])
        writer.writeheader()
        for _ in range(num_data_points):
            suspect = random.choice(suspects)
            variable = random.choice(list(variables.keys()))
            value = random.choice(variables[variable])
            solution_probability = [0.25] * len(suspects)
            solution_probability[suspects.index(suspect)] *= 3.2
            solution = random.choices(suspects, weights=solution_probability)[0]
            writer.writerow({"Case ID": f"C{random.randint(1000, 9999)}", "Suspect": suspect, "Variable": variable, "Value": value, "Description": f"{variable} used as {value}", "Solution": solution})


In [19]:
def build_bayesian_network():
    model = BayesianNetwork([
        ('Weapon', 'Location'),
        ('Location', 'Opportunity'),
        ('Opportunity', 'Motive')] + [(f'Motive', s) for s in suspects])

    cpd_weapon = TabularCPD(variable='Weapon', variable_card=4, values=[[0.25], [0.25], [0.25], [0.25]], state_names={'Weapon': ["Knife", "Gun", "Poison", "Fists"]})
    cpd_location = TabularCPD(variable='Location', variable_card=4, evidence=['Weapon'], evidence_card=[4], values=[[0.25]*4]*4, state_names={'Weapon': ["Knife", "Gun", "Poison", "Fists"], 'Location': ["Living Room", "Kitchen", "Bedroom", "Garden"]})
    cpd_opportunity = TabularCPD(variable='Opportunity', variable_card=3, evidence=['Location'], evidence_card=[4], values=[[0.5, 0.2, 0.2, 0.1], [0.3, 0.4, 0.3, 0.5], [0.2, 0.4, 0.5, 0.4]], state_names={'Location': ["Living Room", "Kitchen", "Bedroom", "Garden"], 'Opportunity': ["High", "Medium", "Low"]})
    cpd_motive = TabularCPD(variable='Motive', variable_card=4, evidence=['Opportunity'], evidence_card=[3], values=[[0.3, 0.1, 0.1], [0.3, 0.4, 0.2], [0.3, 0.4, 0.6], [0.1, 0.1, 0.1]], state_names={'Opportunity': ["High", "Medium", "Low"], 'Motive': ["Jealousy", "Revenge", "Greed", "None"]})

    cpd_john = TabularCPD(variable='John', variable_card=2, evidence=['Motive'], evidence_card=[4],
                      values=[
                          [0.95, 0.1, 0.05, 0.9],
                          [0.05, 0.9, 0.95, 0.1]
                      ], state_names={'Motive': ["Jealousy", "Revenge", "Greed", "None"], 'John': [0, 1]})

    cpd_mary = TabularCPD(variable='Mary', variable_card=2, evidence=['Motive'], evidence_card=[4],
                      values=[
                          [0.05, 0.9, 0.85, 0.1],
                          [0.95, 0.1, 0.15, 0.9]
                      ], state_names={'Motive': ["Jealousy", "Revenge", "Greed", "None"], 'Mary': [0, 1]})

    cpd_peter = TabularCPD(variable='Peter', variable_card=2, evidence=['Motive'], evidence_card=[4],
                       values=[
                           [0.9, 0.1, 0.05, 0.95],
                           [0.1, 0.9, 0.95, 0.05]
                       ], state_names={'Motive': ["Jealousy", "Revenge", "Greed", "None"], 'Peter': [0, 1]})

    cpd_alice = TabularCPD(variable='Alice', variable_card=2, evidence=['Motive'], evidence_card=[4],
                       values=[
                           [0.95, 0.05, 0.1, 0.9],
                           [0.05, 0.95, 0.9, 0.1]
                       ], state_names={'Motive': ["Jealousy", "Revenge", "Greed", "None"], 'Alice': [0, 1]})

    model.add_cpds(cpd_weapon, cpd_location, cpd_opportunity, cpd_motive, cpd_john, cpd_mary, cpd_peter, cpd_alice)
    assert model.check_model()
    return model

In [20]:
def solve_mystery(model, evidence, query_vars):
    if model is None:
        print("Model is not correctly defined.")
        return {}
    inference = VariableElimination(model)
    try:
        results = inference.query(variables=query_vars, evidence=evidence, joint=False)

        formatted_results = {}
        for var in query_vars:
            factor = results[var]
            states = factor.state_names[var]
            probabilities = factor.values
            formatted_results[var] = {state: round(prob, 3) for state, prob in zip(states, probabilities)}
        return formatted_results
    except Exception as e:
        print(f"Error during inference: {e}")
        return {}

In [21]:
if __name__ == "__main__":
    generate_data(1000)
    bayesian_model = build_bayesian_network()
    if bayesian_model:
        evidence = {'Weapon': 'Gun', 'Location': 'Living Room'}
        query_vars = ['John', 'Mary', 'Peter', 'Alice']
        mystery_result = solve_mystery(bayesian_model, evidence, query_vars)
        print("Inference Results:")
        for suspect, probs in mystery_result.items():
            print(f"{suspect}:")
            for state, prob in probs.items():
                print(f"  {state} -> {prob*100}%")

Inference Results:
John:
  0 -> 33.0%
  1 -> 67.0%
Mary:
  0 -> 63.1%
  1 -> 36.9%
Peter:
  0 -> 32.6%
  1 -> 67.4%
Alice:
  0 -> 33.4%
  1 -> 66.60000000000001%
